# IMPORTING LIBRARIES

In [37]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from imutils import paths

# LOADING DATA

In [38]:
training_data = [
    r"E:\aditya\fire_dataset\fire_images",
    r"E:\aditya\fire_dataset\non_fire_images"
]



# Import Libraries:

The code imports NumPy as np and OpenCV (cv2) for image processing.
Define Function load_images(folder):

Creates empty lists images and labels to store images and their corresponding labels.
Iterates through each subfolder in the training data folder.
Initializes a label variable to assign numerical labels to each class.
Iterates through each image file in the subfolders.
Load and Preprocess Images:

Uses OpenCV's imread() function to read each image file as grayscale.
Resizes the grayscale image to 48x48 pixels using OpenCV's resize() function.
Handle Errors Gracefully:

Utilizes a try-except block to handle any errors encountered during image loading.
Store Images and Labels:

Appends the resized grayscale image to the images list.
Appends the corresponding numerical label to the labels list.
Return Arrays:

Converts images and labels lists to NumPy arrays.
Returns the arrays containing the images and their labels.

In [39]:
def load_images(training_data):
    images = []
    labels = []
    for i in range(len(training_data)):
        folder = training_data[i]
        label = i
        for filename in os.listdir(folder):
            try:
                img = cv2.imread(os.path.join(folder,filename),cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (48,48))
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image {os.path.join(folder, filename)}: {e}")
    return np.array(images), np.array(labels)


# Import Libraries:

The code imports necessary libraries for loading, splitting, and preprocessing the data.
Define Function load_images(training_data_path):

Creates empty lists to store images and labels.
Gets a list of all files in the training data directory using os.listdir().
Loops through each file name in the list.
Load Images and Labels:

Reads each image file using cv2.imread() and converts it into an array of pixels.
Appends these pixel arrays (images) into the images list.
Extracts digits from each file name using regex pattern matching and appends them into the labels list.
Return Data:

Returns both lists (images and labels) as the output of the function.
Load and Split Data:

Calls the load_images() function with the path to the training dataset.
Assigns the output (lists of images and labels) to variables X_train and y_train.
Split Data into Train and Test Sets:

Uses train_test_split() method from the sklearn library to split the data into random train and test subsets.
Passes four arguments:
i) X_train: Features/inputs data,
ii) y_train: Target/output values,
iii) test_size: Specifies the proportion of the original data kept aside for testing,
iv) random_state: Controls shuffling applied on the data

In [40]:
images, labels = load_images(training_data)
X_train, X_test, y_train, y_test = train_test_split(images,labels, test_size=0.2, random_state=42)
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1).astype('float32')/255
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1).astype('float32')/255

Error loading image E:\aditya\fire_dataset\non_fire_images\non_fire.189.png: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



# The code
converts the training and testing labels into categorical data.
 The input code is too short to provide a detailed and accurate answer. 
To gain deeper insight, try again using a longer piece of code.

In [41]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Import Libraries:

The code imports the necessary libraries, including Keras and TensorFlow, for building a convolutional neural network (CNN) model.
Define Model:

It defines the model using the Sequential() function from Keras, allowing the creation of a sequential stack of layers for the CNN.
Convolutional Layers:

The first layer is a Conv2D layer with 32 filters (feature detectors) of size 3x3 pixels.
It uses the 'relu' activation function to introduce non-linearity.
The input shape is specified as 48x48 pixels with one color channel (grayscale), represented as a matrix of numbers with dimensions 48x48x1.
The second Conv2D layer has 64 filters, also with 'relu' activation function and a kernel size of 3x3 pixels.
Pooling Layer:

A MaxPooling2D layer follows with a pool size of (2,2), reducing the spatial size of the representation while retaining important features from previous layers.
Dropout Regularization:

To prevent overfitting, a Dropout layer is added after pooling with a dropout rate of 0.25.
This means that randomly selected neurons will be ignored during training to improve generalization ability.
Flattening:

The pooled feature maps are flattened into one long vector to be fed into fully connected Dense layers later on.
Dense Layers:

Two Dense layers are added:
The first one has an output dimensionality of 128 units.
The second Dense layer outputs final prediction probabilities for given class labels using the softmax activation function.
The model outputs predictions for the "happy" class.

In [42]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Import Libraries:

Necessary libraries, including Keras and TensorFlow, are imported.
Load and Split Dataset:

The dataset is loaded using Keras's load_data() function, which automatically splits it into training and testing sets.
80% of the data is used for training, and 20% for testing.
Preprocess Data:

Data is reshaped into a format suitable for CNN input.
Each image is converted from an array of pixels to a matrix of pixel values.
Pixel values are normalized between 0 and 1.
Define CNN Model:

CNN model is defined using Sequential().
Three convolutional layers followed by max pooling layers extract features from images.
Features are flattened and passed through two fully connected dense layers.
The output layer has two nodes representing "fire" or "no fire".
Compile Model:

Model is compiled using categorical crossentropy as the loss function since there are more than two classes.
Adam optimizer is specified.
Accuracy is chosen as the metric for evaluation during training.
Train Model:

fit() method is used for training, taking parameters like batch size, epochs, verbosity, and validation data.
The model is trained on the training data.
Save Model:

After training, the model is saved as a .h5 file for future use.
This allows for reuse or deployment without retraining.

In [46]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics =['accuracy'])
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1, validation_data=(X_test,y_test))
model.save("fire_detection_model.keras")  # Use either .keras or .h5 extension

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.8685 - loss: 0.3566 - val_accuracy: 0.8450 - val_loss: 0.4070
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.8902 - loss: 0.2655 - val_accuracy: 0.8600 - val_loss: 0.3760
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.8922 - loss: 0.2440 - val_accuracy: 0.8650 - val_loss: 0.3882
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.9024 - loss: 0.2243 - val_accuracy: 0.8800 - val_loss: 0.3985
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9182 - loss: 0.1765 - val_accuracy: 0.8800 - val_loss: 0.3646
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9306 - loss: 0.1741 - val_accuracy: 0.8600 - val_loss: 0.3656
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9202 - loss: 0.1904 - val_accuracy: 0.8550 - val_loss: 0.4006
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9285 - loss: 0.1683 - val_accuracy: 0.8850 - val_loss: